<a href="https://colab.research.google.com/github/KristinaKonstantynova/Quarto_Report/blob/main/Copy_of_Quarto_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install plotly pandas matplotlib seaborn pycountry


# **# The Hidden Truth: Physical Violence Against Children Around the World**

According to UNICEF, nearly **400 million children under the age of 5** regularly experience violent discipline, including physical punishment and psychological aggression.

> Physical violence against children, though universally condemned, remains prevalent across various parts of the world.  
> Despite progress in human rights, **physical punishment continues to be accepted and practiced** in many regions due to cultural norms, historical contexts, and economic challenges.

This report aims to raise public awareness about this ongoing issue by analyzing global legislation, regional differences, and the evolution of protection laws over time.

We use **UNICEF Indicator 2** as our primary source and complement this analysis with visualizations to provide data-driven insights.


In [ ]:
import pandas as pd
import plotly.express as px
import pycountry

# Load data
df = pd.read_csv("/content/V4 unicef_indicator_2.csv", sep=";", on_bad_lines="skip")

# Filter for relevant indicator and "Total" population
filtered = df[
    (df["sex"] == "Total") &
    (df["indicator"].str.contains("physical punishment", case=False, na=False)) &
    (~df["obs_value"].isna())
][["country", "obs_value", "time_period"]].copy()

# Match country names to Plotly-compatible names
def match_country(name):
    try:
        return pycountry.countries.lookup(name).name
    except:
        return name

filtered["country_clean"] = filtered["country"].apply(match_country)

# Convert year to string (for animation frame)
filtered["year"] = filtered["time_period"].astype(str)

# Create choropleth with animation
fig = px.choropleth(
    filtered,
    locations="country_clean",
    locationmode="country names",
    color="obs_value",
    animation_frame="year",
    color_continuous_scale=px.colors.sequential.Reds,
    range_color=[0, 100],
    title="🌍 Evolution of Global Support for Physical Punishment (UNICEF Indicator 2)",
    labels={"obs_value": "% Support"},
    hover_name="country_clean",
    custom_data=["obs_value"]
)

# Customize hover
fig.update_traces(
    hovertemplate="<b>%{hovertext}</b><br>Support: %{customdata[0]:.1f}%<extra></extra>"
)

# Style layout
fig.update_layout(
    title_font_size=18,
    margin=dict(t=50, r=10, b=20, l=10),
    coloraxis_colorbar=dict(
        title="Support (%)",
        tickfont=dict(size=10)
    ),
    annotations=[
        dict(
            x=0.5, y=-0.18,
            xref='paper', yref='paper',
            text="Source: UNICEF Indicator 2 — Adults supporting physical punishment<br>Animated by year",
            showarrow=False,
            font=dict(size=11),
            xanchor='center'
        )
    ]
)

# Use natural projection and show borders
fig.update_geos(
    projection_type="natural earth",
    showcountries=True,
    countrycolor="lightgray",
    showframe=False
)

fig.show()


Across the world, physical punishment remains deeply ingrained in many cultures.  
While some regions have made progress in reducing its acceptance, others continue to grapple with widespread support for corporal punishment.

**This map shows just how diverse the situation is**:  
🌍 **Oceania and parts of Africa** still show **high levels of support**,  
while **Europe has made remarkable progress**, with much lower acceptance.


In [ ]:
import plotly.express as px

# Prepare data again (just in case)
continent_df = indicator[
    (indicator["sex"] == "Total") &
    (indicator["indicator"].str.contains("physical punishment", case=False, na=False)) &
    (~indicator["obs_value"].isna()) &
    (~indicator["continents"].isna())
]

# Group and average support by continent
continent_avg = continent_df.groupby("continents", as_index=False)["obs_value"].mean()
continent_avg = continent_avg.rename(columns={"obs_value": "support_pct", "continents": "Continent"})
continent_avg["support_pct"] = continent_avg["support_pct"].round(1)

# Sort by highest support
continent_avg = continent_avg.sort_values("support_pct", ascending=False)

# Plot with Plotly
fig = px.bar(
    continent_avg,
    x="support_pct",
    y="Continent",
    orientation="h",
    color="support_pct",
    color_continuous_scale="Reds",
    labels={"support_pct": "Physical Punishment Support (%)"},
    title="Where People Most Support Physical Punishment (by Continent)",
    text="support_pct"
)

# Customize hover and text
fig.update_traces(
    texttemplate="%{text}%",
    hovertemplate="<b>%{y}</b><br>Support: %{x:.1f}%<extra></extra>",
    marker_line_color='black',
    marker_line_width=0.7
)

# Layout tweaks
fig.update_layout(
    title_font_size=18,
    coloraxis_showscale=False,
    xaxis_title="Physical Punishment Support (%)",
    yaxis_title=None,
    margin=dict(t=70, r=40, b=50, l=90),
    height=500
)

fig.show()


Looking at the continents, we see stark differences in attitudes toward physical punishment.

🌍 **Oceania**, for instance, shows consistently high levels of support for physical punishment, despite modern living conditions.  
🌍 **Africa** also continues to show higher levels of acceptance, driven in part by cultural traditions and economic factors.  
🌍 In contrast, **Europe** stands out with a significant decline in support — a shift largely due to legal reforms and a cultural movement toward non-violent child-rearing.


In [ ]:
metadata = pd.read_csv("/content/unicef_metadata.csv", sep=",", quotechar='"', on_bad_lines="skip")
metadata.columns = metadata.columns.str.strip()  # Clean column names


In [ ]:
print(metadata.columns.tolist())


['country', 'alpha_2_code', 'alpha_3_code', 'numeric_code', 'year', 'Population, total', 'GDP per capita (constant 2015 US$)', 'GNI (current US$)', 'Inflation, consumer prices (annual %)', 'Life expectancy at birth, total (years)', 'Military expenditure (% of GDP)', 'Fossil fuel energy consumption (% of total)', 'GDP growth (annual %)', 'Birth rate, crude (per 1,000 people)', 'Hospital beds (per 1,000 people)']


In [ ]:
gdp_column = "GDP per capita (constant 2015 US$)"

gdp_df = metadata[["country", gdp_column]].copy()
gdp_df = gdp_df.rename(columns={gdp_column: "gdp_per_capita"})


NameError: name 'metadata' is not defined

In [ ]:
import plotly.express as px

# Reuse the merged and cleaned scatter_df from before
# Make sure it's already created with: country, support_pct, gdp_per_capita, continents

fig = px.scatter(
    scatter_df,
    x="gdp_per_capita",
    y="support_pct",
    color="continents",
    hover_name="country",
    size_max=15,
    title="GDP per Capita vs. Support for Physical Punishment",
    labels={
        "gdp_per_capita": "GDP per Capita (USD)",
        "support_pct": "% Supporting Physical Punishment",
        "continents": "Continent"
    },
    custom_data=["gdp_per_capita", "support_pct", "continents"],
    template="plotly_white"
)

# Custom hover formatting
fig.update_traces(
    marker=dict(size=12, line=dict(width=1, color='DarkSlateGrey')),
    hovertemplate=(
        "<b>%{hovertext}</b><br>" +
        "Continent: %{customdata[2]}<br>" +
        "GDP per Capita: $%{customdata[0]:,.0f}<br>" +
        "Support: %{customdata[1]:.1f}%<extra></extra>"
    )
)

# Layout tweaks
fig.update_layout(
    title_font_size=18,
    margin=dict(t=70, r=20, b=60, l=60),
    legend_title_text="Continent",
    hoverlabel=dict(font_size=12),
    annotations=[dict(
        text="Source: UNICEF Indicator 2 + Metadata",
        x=0.5, y=-0.25, xref='paper', yref='paper',
        showarrow=False, font=dict(size=11),
        xanchor='center'
    )]
)

fig.show()


There is a clear connection between economic conditions and support for physical punishment.  
In wealthier countries, where economic stability and education are more accessible, support for physical punishment tends to be lower.  
In contrast, poorer regions like **Africa** still see high levels of corporal punishment, driven by poverty, lack of education, and social stress.

However, some wealthy countries, such as those in **Oceania**, still show significant support for physical punishment — suggesting that cultural norms play a strong role alongside economic factors.  
This underscores the **complex relationship between economic growth, cultural traditions, and disciplinary practices**.


In [ ]:
import plotly.express as px

# Filter and clean Indicator 2 data
ts_df = indicator[
    (indicator["sex"] == "Total") &
    (indicator["indicator"].str.contains("physical punishment", case=False, na=False)) &
    (~indicator["obs_value"].isna()) &
    (~indicator["continents"].isna())
].copy()

# Group by year and continent
ts_grouped = ts_df.groupby(["time_period", "continents"], as_index=False)["obs_value"].mean()
ts_grouped = ts_grouped.rename(columns={"obs_value": "support_pct", "time_period": "Year", "continents": "Continent"})

# Create interactive time series line chart
fig = px.line(
    ts_grouped,
    x="Year",
    y="support_pct",
    color="Continent",
    markers=True,
    template="plotly_white",
    title="Trends in Physical Punishment Support by Continent",
    labels={"support_pct": "Physical Punishment Support (%)"},
    hover_name="Continent",
)

# Customize tooltips
fig.update_traces(
    hovertemplate=(
        "<b>%{hovertext}</b><br>" +
        "Year: %{x}<br>" +
        "Support: %{y:.1f}%<extra></extra>"
    ),
    line=dict(width=2)
)

# Customize layout
fig.update_layout(
    title_font_size=18,
    xaxis_title="Year",
    yaxis_title="Physical Punishment Support (%)",
    legend_title="Continent",
    hoverlabel=dict(font_size=12),
    margin=dict(t=60, r=20, b=60, l=60),
    annotations=[
        dict(
            text="Source: UNICEF Indicator 2 — Average per Year by Continent",
            x=0.5, y=-0.25, xref='paper', yref='paper',
            showarrow=False, font=dict(size=11),
            xanchor='center'
        )
    ]
)

fig.show()


Looking at trends over time, we can see that attitudes have changed in many regions.  
While some countries — particularly in **Europe** — have consistently reduced support, others like those in **Oceania** and **Africa** remain high.

Though global trends have evolved over time, the year **2020** stands out, particularly in Oceania.  
The sharp rise in support for physical punishment — jumping from **38% to 86%** — points to a deeper societal change, likely influenced by the **COVID-19 pandemic** and the stresses that came with it.

This shift illustrates how **external pressures** can force societies to revisit long-standing practices and attitudes toward discipline.


<h2 style="text-align: center; color: #8B0000;">BUT WHY DO SOME COUNTRIES STILL SUPPORT PHYSICAL PUNISHMENT?</h2>

<div style="display: flex; justify-content: space-around; text-align: center; font-size: 16px; flex-wrap: wrap;">

  <details style="margin: 10px; width: 22%;">
    <summary>🏛️ <b>Cultural Traditions and Norms</b></summary>
    <p style="text-align: left;">
      In many cultures, physical punishment is deeply rooted in traditional views of discipline.
      It is often seen as a sign of strong parenting and is normalized through generations.
    </p>
  </details>

  <details style="margin: 10px; width: 22%;">
    <summary>💰 <b>Economic and Social Factors</b></summary>
    <p style="text-align: left;">
      Poverty, social stress, and lack of educational access increase frustration and normalize violence.
      Parents under economic strain may resort to stricter, harsher methods of control.
    </p>
  </details>

  <details style="margin: 10px; width: 22%;">
    <summary>⚖️ <b>Legal and Institutional Factors</b></summary>
    <p style="text-align: left;">
      In some countries, there are no laws banning physical punishment at home or in schools.
      Even where laws exist, they may not be enforced, allowing the practice to persist unchecked.
    </p>
  </details>

  <details style="margin: 10px; width: 22%;">
    <summary>✝️ <b>Religious Beliefs</b></summary>
    <p style="text-align: left;">
      Certain interpretations of religious texts promote physical discipline as a moral or righteous act.
      This creates strong resistance to legal reforms or cultural change.
    </p>
  </details>

</div>


<h2 style="color: #b23a48; font-weight: bold; text-align: right;">HOW TO CHANGE IT?</h2>

<table style="width:100%;">
  <tr>
    <td style="width:45%; vertical-align: top;">
      <img src="/content/d143f574-96df-4e7b-a0a8-273f4f31107f.png" width="100%">
    </td>
    <td style="width:55%; padding-left: 25px;">
      <p><span style="color:#b23a48; font-weight: bold;">Physical punishment</span> is not a thing of the past; it remains a global issue shaped by a complex interplay of <b>culture, economics, and historical practices</b>.</p>

      <p>While some regions, particularly in Europe, are seeing progress through legal reforms and cultural shifts, others—especially in Oceania and Africa—still face significant challenges. The path forward requires <b>legal reforms</b>, <b>cultural change</b>, and <b>increased awareness</b> to break the cycle of violence.</p>

      <a href="https://www.unicef.ie/donate/?c=694#1" target="_blank">
        <img src="https://i.imgur.com/nBaQgQe.png" alt="Donate Button" width="160" style="margin-top: 15px;">
      </a>
    </td>
  </tr>
</table>


<div style="text-align: center;">
  <img src="/content/palestine.jpg" width="50%">
  <p style="font-size: 15px; color: #555; margin-top: 10px;"><i>“The silence of pain is often louder than words.”</i></p>
</div>
